In [1]:
import pandas as pd
import numpy as np  
import torch 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

#stuff to do:
#hyperparameter tune (maybe use gridsearch implementation)
#Check and modify feature importance
    #import matplotlib.pyplot as plt
    #Plot feature importance
    #xgb.plot_importance(model)
    #plt.show()
#incorporate feature engineering

In [2]:
df = pd.read_csv("combined_league_matches.csv")
champion_ids = [col for col in df.columns if 'champion' in col]
all_games = pd.concat([df[col] for col in champion_ids]) 
games_per_champ = all_games.value_counts() #maybe useful, find total number of games per champ id


encoder = LabelEncoder()

for col in ['red_champion_0', 'red_champion_1', 'red_champion_2', 'red_champion_3', 'red_champion_4',
            'blue_champion_0', 'blue_champion_1', 'blue_champion_2', 'blue_champion_3', 'blue_champion_4']:
    df[col] = encoder.fit_transform(df[col])

df['target'] = df['winner'].apply(lambda x: 1 if x == 'blue' else 0) #blueside win if 1
df = df.drop(columns=['match_id', 'winner'])

#if we want to test without mastery just uncomment below
#df = df.drop(columns=['red_mastery_0', 'red_mastery_1', 'red_mastery_2', 'red_mastery_3', 'red_mastery_4',
#            'blue_mastery_0', 'blue_mastery_1', 'blue_mastery_2', 'blue_mastery_3', 'blue_mastery_4'])

#scale all mastery values
mastery_columns = [
    'red_mastery_0', 'red_mastery_1', 'red_mastery_2', 'red_mastery_3', 'red_mastery_4',
    'blue_mastery_0', 'blue_mastery_1', 'blue_mastery_2', 'blue_mastery_3', 'blue_mastery_4'
]

scaler = StandardScaler() #can also try minmaxscaler and keep within certain range

df[mastery_columns] = scaler.fit_transform(df[mastery_columns])

In [3]:
X = df.drop(columns=['target'])
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [4]:
model = XGBClassifier(
    n_estimators=100,     # Number of trees
    learning_rate=0.1,    # Step size shrinkage
    max_depth=6,          # Maximum depth of trees
    subsample=0.8,        # Subsample ratio of the training instance
    colsample_bytree=0.8, # Subsample ratio of columns when constructing trees
)

model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [5]:
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print(classification_report(y_test, y_pred))

Accuracy: 0.57
              precision    recall  f1-score   support

           0       0.57      0.45      0.50      2233
           1       0.57      0.68      0.62      2426

    accuracy                           0.57      4659
   macro avg       0.57      0.57      0.56      4659
weighted avg       0.57      0.57      0.56      4659



Feature Engineering Ideas:

In [ ]:
#Aggregate mastery level for each team  aka average and max mastery of each team

#Todo:


In [ ]:
#Compute Mastery Difference for each team

#Todo:


In [ ]:
#Harder to do -> encode relationships between champs (how well certain champs counter or synergize)

In [ ]:
#Encode Team Composition represent red and blue teams as sets of characters

In [ ]:
#Normalize anything implemented above using Standard Scaler (Look at basicrandomforest notebook for example)

In [ ]:
#Maybe try one_hot_encoding

In [ ]:
#After above try feature engineering using XGBoost importance 
#Use df.corr to remove redundant features.